INTRODUCCIÓN
==========================================

Regresión logística conp pytorch sobre el dataset Heart Failure Clinical Records (2020). <br>
Fuente: https://archive.ics.uci.edu/dataset/519/heart+failure+clinical+records


CONCEPTOS BÁSICOS
==========================================

Para entender mejor de qué va el trabajo es necesario conocer antes una serie de conceptos sobre los recursos y otras herramientas usadas:
- <b>Redes neuronales</b>: Una red neuronal es un modelo computacional inspirado en el funcionamiento del cerebro humano. Está compuesta por un conjunto de nodos o "neuronas" interconectadas que procesan información. Aprenden a realizar tareas al considerar ejemplos, generalmente sin ser programadas con reglas específicas para la tarea.
- <b>Pandas</b>: Biblioteca de Python que proporciona estructuras de datos y herramientas de análisis de datos flexibles y eficientes. Es fundamental para realizar tareas de manipulación de datos y limpieza.
- <b>Pytorch</b>: Es una biblioteca de aprendizaje automático de código abierto para Python, desarrollada principalmente por el grupo de investigación de inteligencia artificial de Facebook(Meta). Proporciona dos características de alto nivel: operaciones tensoriales con soporte para GPU y redes neuronales profundas construidas sobre un sistema de autograd (cálculo automático de gradientes) flexible.
- <b>Tensores</b>: Un tensor es una estructura de datos generalizada que es una extensión de matrices multidimensionales. En el contexto de PyTorch y otras bibliotecas de aprendizaje profundo, los tensores son la estructura de datos principal con la que se trabaja.
Los tensores son similares a los arrays de NumPy, pero también tienen soporte para GPU y otras características que los hacen adecuados para el cálculo de redes neuronales.
- <b>Función de perdida</b>: También conocida como función de coste, es una fórmula matemática que mide cuán bien el modelo está realizando su tarea, ya sea la clasificación, la regresión, etc. Las ha de varios tipos, según la naturaleza de los datos. Algunos ejemplos son BCELoss o MSELoss.
- <b>Optimizador</b>: Es un algoritmo o método que se utiliza para ajustar los parámetros de un modelo con el objetivo de minimizar la función de pérdida. Como la función de perdida también hay multitud y debemos buscar el más adecuado para nuestra carga de trabajo. Algunos ejemplos son Adam, adagrad, Rmsprop...
- <b>Train y test</b>: Sets (conjunto de datos) de entrenamiento y de validación (test). Para entrenar un modelo es necesario dividir nuestros datos en dos sets, uno de entrenamiento sobre el que realizaremos todo el trabajo y otro de test con el que comprobaremos la validad de los datos. Deben ser distintos.
- <b>Normalización de datos</b>: El objetivo de la normalización es cambiar los valores de las columnas numéricas en el conjunto de datos a una escala común, sin distorsionar las diferencias en los rangos de valores o perder información.
- <b>Epoch</b>: Iteracciones de entrenamiento, ciclos. Debemos jugar con este parametro númerico y otros para poder entrenar nuestro módelo de forma óptima.
- <b>Tasa de aprendizaje</b>: Otro parametro de entrenamiento que define cuán rapido aprende nuestro modelo. Se usan valores del estilo 0.1, 0.01... También debemos probar bien ya que no siempre un valor más alto nos lleva a un entrenamiento más rapido y efectivo.
- <b>Backward pass</b>: Durante este paso, se calculan los gradientes de la función de pérdida con respecto a los parámetros del modelo (pesos y sesgos), utilizando un algoritmo llamado retropropagación.

